In [2]:
%pip install kinopoisk-api-unofficial-client

     -------------------------------------- 50.7/50.7 kB 861.4 kB/s eta 0:00:00
     ---------------------------------------- 62.3/62.3 kB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 93.1/93.1 kB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 48.9/48.9 kB ? eta 0:00:00
   ---------------------------------------- 143.8/143.8 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
  Attempting uninstall: charset-normalizer
    Found existing installation: charset-normalizer 3.3.1
    Uninstalling charset-normalizer-3.3.1:
      Successfully uninstalled charset-normalizer-3.3.1
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
Note: you may need to restart the kernel to use updated packages.


In [103]:
from kinopoisk_unofficial.kinopoisk_api_client import KinopoiskApiClient
from kinopoisk_unofficial.request.films.film_request import FilmRequest

from kinopoisk_unofficial.request.films.search_by_keyword_request import SearchByKeywordRequest
from kinopoisk_unofficial.request.films.seasons_request import SeasonsRequest

# для проверки возрастных ограничений
import re 

api_client = KinopoiskApiClient("14df4088-0c1e-475f-8295-2330a36a15ab")

name_request = 'Вестворлд' # input()

# get film id by name request:

request = SearchByKeywordRequest(name_request)
response = api_client.films.send_search_by_keyword_request(request)
request_film_id = response.films[0].film_id
id = FilmRequest(request_film_id)

# get film info by id:

film_response = api_client.films.send_film_request(id)

# check if our request is series

film_is_series = film_response.film.serial

# по дефолту количество сезонов None
seasons_cnt = None

# get series info by id

if film_is_series:
    try:
        series_id = SeasonsRequest(request_film_id)
        series_response = api_client.films.send_seasons_request(series_id)

        seasons_cnt = series_response.total
        # возьмем количество эпизодов из первого сезона
        for episode in series_response.items[0].episodes:
            episodes_cnt = episode.episode_number
    except:
        client_message = "Не удалось получить количество сезонов"


film_name = film_response.film.name_ru

film_types = {"FILM" : "Фильм", "TV_SERIES" : "Сериал", "MINI_SERIES" : "Видео", "TV_SHOW" : "ТВ шоу", "VIDEO" : "Видео"}
film_type = film_types[film_response.film.type.name]

film_kinopoisk_rating = film_response.film.rating_kinopoisk
film_imbd_rating = film_response.film.rating_imdb
film_year = film_response.film.year
film_length = film_response.film.film_length

film_genres = []
for genre in film_response.film.genres:
    film_genres.append(genre.genre)

film_countries = []
for country in film_response.film.countries:
    film_countries.append(country.country)

film_description = film_response.film.description
film_short_description = film_response.film.short_description
film_poster_url = film_response.film.poster_url

film_age = int(re.findall(r'\d+', film_response.film.rating_age_limits)[0])
film_url = film_response.film.web_url



In [105]:
class Film:
    def __init__(self, name : str, type : str, year : int, rating : float, duration : int, poster_url : str, genre : list, country : list, description: str, short_description : str, age : int, site : str, site_url : str, is_series : bool, seasons=None):
        self.name = name
        self.type = type
        self.year = year
        self.rating = rating
        self.duration = duration # фильма/серии
        self.poster_url = poster_url
        self.genre = genre
        self.country = country
        self.description = description
        self.short_description = short_description
        self.age = age
        self.site = site
        self.site_url = site_url
        self.is_series = is_series
        self.seasons = seasons

film = Film(film_name, film_type, film_year, film_kinopoisk_rating, film_length, film_poster_url, film_genres, film_countries, film_description, film_short_description, film_age, "kinopoisk", film_url, film_is_series, seasons_cnt)

In [106]:
print(f'{film.name}, {film.year}')
print(film.type)
print(f'Рейтинг: {film.rating}')
if film.is_series:
    print(f'Продолжительность серии: {film.duration}')
    if film.seasons:
        print(f'Количество сезонов: {film.seasons}')
    else:
        print(client_message)
else:
    print(f'Продолжительность: {film.duration}')
print(film.poster_url)
print(', '.join(film.genre))
print(', '.join(film.country))
print(f'{film.age}+')
print(film.description)
print(film.site_url)

Мир Дикого Запада, 2016
Сериал
Рейтинг: 7.9
Продолжительность серии: 62
Количество сезонов: 4
https://kinopoiskapiunofficial.tech/images/posters/kp/195523.jpg
драма, детектив, фантастика
США
18+
В футуристическом парке развлечений «Мир Дикого Запада» специально сконструированные андроиды выполняют любые прихоти посетителей, чтобы те чувствовали безнаказанность и полную свободу действий. Если робота убили — не беда, техники его починят, сотрут память и снова поставят в строй, навстречу новому дню и новым людским прихотям. Но оказывается, что далеко не все роботы теряют воспоминания.
https://www.kinopoisk.ru/film/195523/


![](https://kinopoiskapiunofficial.tech/images/posters/kp/462762.jpg)